In [29]:
import pandas as pd

data_path = r"C:\Users\tpapa\OneDrive\Desktop\PhD Work\Portfolio-Optimization-using-LLMs-and-RL\data\FNSPID_portfolio_finbert_sentiment_score.csv"

df_with_sentiment = pd.read_csv(data_path)


df_with_sentiment.head()

,Date,Ticker,sentiment,score
0,2009-06-22,KO,Neutral,0.910568
1,2009-08-07,AXP,Neutral,0.999998
2,2009-08-10,AXP,Neutral,0.999977
3,2009-08-10,KO,Positive,0.804819
4,2009-08-13,CVS,Neutral,0.999984


In [30]:
import numpy as np

conditions = [
    (df_with_sentiment['sentiment'] == 'Positive') & (df_with_sentiment['score'] > 0.6),
    (df_with_sentiment['sentiment'] == 'Positive') & (df_with_sentiment['score'] <= 0.6),
    (df_with_sentiment['sentiment'] == 'Negative') & (df_with_sentiment['score'] > 0.6),
    (df_with_sentiment['sentiment'] == 'Negative') & (df_with_sentiment['score'] <= 0.6),
    (df_with_sentiment['sentiment'] == 'Neutral')
]

values = [1, 0.5, -1, -0.5, 0]

df_with_sentiment['company_sentiment'] = np.select(conditions, values)

df_with_sentiment.head()

,Date,Ticker,sentiment,score,company_sentiment
0,2009-06-22,KO,Neutral,0.910568,0.0
1,2009-08-07,AXP,Neutral,0.999998,0.0
2,2009-08-10,AXP,Neutral,0.999977,0.0
3,2009-08-10,KO,Positive,0.804819,1.0
4,2009-08-13,CVS,Neutral,0.999984,0.0


In [31]:
ticker_to_sector = {
    'ABT': 'Healthcare', 'UNH': 'Healthcare', 'JNJ': 'Healthcare', 'CVS': 'Healthcare',
    'AMZN': 'Consumer Discretionary', 'MCD': 'Consumer Discretionary', 'F': 'Consumer Discretionary', 'GM': 'Consumer Discretionary',
    'BLK': 'Financial', 'BRK.B': 'Financial', 'AXP': 'Financial', 'JPM': 'Financial',
    'XOM': 'Energy', 'CVX': 'Energy', 'DVN': 'Energy', 'CTRA': 'Energy',
    'NVDA': 'Tech', 'AAPL': 'Tech', 'AMD': 'Tech', 'CSCO': 'Tech',
    'T': 'Communications', 'CMCSA': 'Communications', 'TMUS': 'Communications', 'VZ': 'Communications',
    'KO': 'Consumer Staples', 'PG': 'Consumer Staples', 'COST': 'Consumer Staples', 'KMB': 'Consumer Staples',
    'GE': 'Industrials', 'MMM': 'Industrials', 'DAL': 'Industrials', 'BA': 'Industrials'
}

df_with_sentiment['Sector'] = df_with_sentiment['Ticker'].map(ticker_to_sector)

df_with_sentiment.head()

,Date,Ticker,sentiment,score,company_sentiment,Sector
0,2009-06-22,KO,Neutral,0.910568,0.0,Consumer Staples
1,2009-08-07,AXP,Neutral,0.999998,0.0,Financial
2,2009-08-10,AXP,Neutral,0.999977,0.0,Financial
3,2009-08-10,KO,Positive,0.804819,1.0,Consumer Staples
4,2009-08-13,CVS,Neutral,0.999984,0.0,Healthcare


In [32]:
main_companies = {
    'Healthcare': 'ABT',
    'Consumer Discretionary': 'AMZN',
    'Financial': 'BLK',
    'Energy': 'XOM',
    'Tech': 'NVDA',
    'Communications': 'T',
    'Consumer Staples': 'KO',
    'Industrials': 'GE'
}

#df_with_sentiment['Date'] = pd.to_datetime(df_with_sentiment['Date'], errors='coerce').dt.normalize()
#df_with_sentiment['Date'] = pd.to_datetime(df_with_sentiment['Date'].str[:10], format='%Y-%m-%d')

df_dummies = df_with_sentiment[~df_with_sentiment.apply(
    lambda row: row['Ticker'] == main_companies.get(row['Sector']), axis=1)]

# Get average of sector sentiment to create sector_sentiment 
sector_avg_sentiment = df_dummies.groupby(['Sector', 'Date'])['company_sentiment'].mean().reset_index()
sector_avg_sentiment.rename(columns={'company_sentiment': 'sector_sentiment'}, inplace=True)

# Keep "main" companies here, these will be my portfolio.
df_main = df_with_sentiment[df_with_sentiment.apply(
    lambda row: row['Ticker'] == main_companies.get(row['Sector']), axis=1)]

# Merge on sector and fillna with 0. Not true that lack of sentiment shows neutral sentiment but this is a simplified example.
df_main = df_main.merge(sector_avg_sentiment, on=['Sector', 'Date'], how='left')
df_main['sector_sentiment'] = df_main['sector_sentiment'].fillna(0.0)

df_main.shape

(16576, 7)

In [ ]:
import pandas as pd
import yfinance as yf
from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer
from finrl.config import INDICATORS

main_tickers = list(main_companies.values())

EXPANDED_START_DATE = "2008-01-01"
START_DATE = "2009-01-01"
END_DATE = "2023-12-31"

df = YahooDownloader(start_date = EXPANDED_START_DATE,
                     end_date = END_DATE,
                     ticker_list= main_tickers).fetch_data()

print(df.shape)

df = FeatureEngineer(use_technical_indicator=True,
                     tech_indicator_list = INDICATORS,
                     use_turbulence = False,
                     user_defined_feature = False).preprocess_data(df.copy())

print(df.shape)

df = df.sort_values(by=['tic', 'date'], ignore_index=True)

df['close_logdiff'] = df.groupby('tic')['close'].transform(lambda x: np.log(x) - np.log(x.shift(1)))
df['close_30_sma_logdiff'] = df.groupby('tic')['close_30_sma'].transform(lambda x: np.log(x) - np.log(x.shift(1)))
df['close_60_sma_logdiff'] = df.groupby('tic')['close_60_sma'].transform(lambda x: np.log(x) - np.log(x.shift(1)))

# Calculate the covariance matrix of the logarithmic returns and add as feature
logdiff_pivot = df.pivot(index='date', columns='tic', values='close_logdiff')

rolling_covs = logdiff_pivot.rolling(window=252).cov(pairwise=True)

cov_features = []

for date in logdiff_pivot.index[251:]:  
    cov_matrix = rolling_covs.loc[date]
    
    for ticker in logdiff_pivot.columns:
        cov_row = cov_matrix.loc[ticker]  
        cov_row.index = [f'cov_{t}' for t in cov_row.index]
        
        cov_features.append({
            'date': date,
            'tic': ticker,
            **cov_row.to_dict()
        })

cov_df = pd.DataFrame(cov_features)

# Merge
df = df.merge(cov_df, on=['date', 'tic'], how='left')

df.drop(['high', 'low', 'open', 'volume', 'day'],
        axis = 'columns',
        inplace = True)

df = df[df["date"] >= START_DATE].reset_index(drop=True)

df = df.sort_values(by=['date']).reset_index(drop=True)

print(df.shape)

df.head(8)

(32216, 8)
Successfully added technical indicators
(32216, 16)
(30192, 22)


,date,close,tic,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,...,close_30_sma_logdiff,close_60_sma_logdiff,cov_ABT,cov_AMZN,cov_BLK,cov_GE,cov_KO,cov_NVDA,cov_T,cov_XOM
0,2009-01-02,17.675545,ABT,0.003332,17.762737,16.779159,50.088584,96.529041,24.299580,17.205826,...,-0.001923,-0.000598,0.000363,0.000325,0.000440,0.000338,0.000241,0.000340,0.000333,0.000350
1,2009-01-02,6.600315,T,0.062455,6.618740,6.048167,53.674745,121.227361,5.384956,6.242854,...,0.003420,0.002602,0.000333,0.000717,0.000833,0.000598,0.000376,0.000759,0.000805,0.000663
2,2009-01-02,44.505531,XOM,0.416805,45.354465,40.148059,53.950528,100.277274,0.441526,42.429993,...,0.002277,0.001342,0.000350,0.000755,0.000779,0.000629,0.000403,0.000889,0.000663,0.001048
3,2009-01-02,2.718000,AMZN,0.059828,2.715886,2.362514,51.654500,88.278119,1.201570,2.371067,...,0.011254,-0.001406,0.000325,0.001722,0.001192,0.000921,0.000459,0.001075,0.000717,0.000755
4,2009-01-02,0.199677,NVDA,0.002846,0.212585,0.158617,51.461837,87.354806,2.289497,0.176462,...,0.008349,0.003197,0.000340,0.001075,0.001246,0.000898,0.000492,0.002977,0.000759,0.000889
5,2009-01-02,91.836182,BLK,1.588929,93.819390,75.821400,51.021156,110.975509,1.376431,81.385317,...,0.009109,-0.004060,0.000440,0.001192,0.002127,0.001021,0.000490,0.001246,0.000833,0.000779
6,2009-01-02,13.855596,KO,0.014900,13.999604,13.167621,50.387807,84.729305,0.411489,13.507794,...,0.002071,-0.001117,0.000241,0.000459,0.000490,0.000338,0.000494,0.000492,0.000376,0.000403
7,2009-01-02,56.576824,GE,-0.827988,60.642696,49.963679,47.556142,43.223431,12.380352,53.881468,...,0.002696,-0.002721,0.000338,0.000921,0.001021,0.001247,0.000338,0.000898,0.000598,0.000629


In [60]:
df_main = df_main.rename({"Date":"date",
                         "Ticker": 'tic'},
                          axis = 1)

df_full = pd.merge(
    df,
    df_main[['date', 'tic', 'company_sentiment', 'sector_sentiment']],
    on = ['date', 'tic'],
    how = 'left'
)

df_full['company_sentiment'] = df_full['company_sentiment'].fillna(0.0)
df_full['sector_sentiment'] = df_full['sector_sentiment'].fillna(0.0)

print(df_full.shape)

df_full.head()

(30192, 24)


,date,close,tic,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,...,cov_ABT,cov_AMZN,cov_BLK,cov_GE,cov_KO,cov_NVDA,cov_T,cov_XOM,company_sentiment,sector_sentiment
0,2009-01-02,17.675545,ABT,0.003332,17.762737,16.779159,50.088584,96.529041,24.299580,17.205826,...,0.000363,0.000325,0.000440,0.000338,0.000241,0.000340,0.000333,0.000350,0.0,0.0
1,2009-01-02,6.600315,T,0.062455,6.618740,6.048167,53.674745,121.227361,5.384956,6.242854,...,0.000333,0.000717,0.000833,0.000598,0.000376,0.000759,0.000805,0.000663,0.0,0.0
2,2009-01-02,44.505531,XOM,0.416805,45.354465,40.148059,53.950528,100.277274,0.441526,42.429993,...,0.000350,0.000755,0.000779,0.000629,0.000403,0.000889,0.000663,0.001048,0.0,0.0
3,2009-01-02,2.718000,AMZN,0.059828,2.715886,2.362514,51.654500,88.278119,1.201570,2.371067,...,0.000325,0.001722,0.001192,0.000921,0.000459,0.001075,0.000717,0.000755,0.0,0.0
4,2009-01-02,0.199677,NVDA,0.002846,0.212585,0.158617,51.461837,87.354806,2.289497,0.176462,...,0.000340,0.001075,0.001246,0.000898,0.000492,0.002977,0.000759,0.000889,0.0,0.0


In [61]:
df_full.to_pickle(r"C:\Users\tpapa\OneDrive\Desktop\PhD Work\Portfolio-Optimization-using-LLMs-and-RL\data\full_data_stock_price_tech_ind_sentiment.pkl")